In [1]:
import tensorflow as tf
import random

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Conv Layer 1

In [4]:
learning_rate = 0.01
training_epochs = 15
batch_size = 100

In [5]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28,28,1])
Y = tf.placeholder(tf.float32, [None, 10])

In [6]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))

In [7]:
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [8]:
print(L1)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


 # Conv Layer 2

In [10]:
W2 = tf.Variable(tf.random_normal([3,3,32,64],  stddev=0.01))

In [11]:
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [12]:
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])

# Final FC

In [15]:
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

In [16]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
print("Learning started. It takes sometime.")

Learning started. It takes sometime.


In [19]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/ batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
    
print('learning finished')

Epoch: 0001 cost =  0.154969713
Epoch: 0002 cost =  0.054102311
Epoch: 0003 cost =  0.043495051
Epoch: 0004 cost =  0.036152485
Epoch: 0005 cost =  0.030769446
Epoch: 0006 cost =  0.028541241
Epoch: 0007 cost =  0.027956295
Epoch: 0008 cost =  0.022836915
Epoch: 0009 cost =  0.022333560
Epoch: 0010 cost =  0.020597457
Epoch: 0011 cost =  0.019824998
Epoch: 0012 cost =  0.021361995
Epoch: 0013 cost =  0.017115843
Epoch: 0014 cost =  0.025050731
Epoch: 0015 cost =  0.023914170
learning finished


In [20]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy : ', sess.run(accuracy, feed_dict={X:mnist.test.images, Y: mnist.test.labels}))

Accuracy :  0.9836


In [21]:
r = random.randint(0, mnist.test.num_examples - 1)
print("Label : ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prediction : ", sess.run(tf.argmax(logits,1), feed_dictct={X:mnist.test.images[r:r+1]}))

Label :  [9]
Prediction :  [9]
